In [3]:
import os
import pandas as pd

In [4]:
csv_path = '../raw_data'
cleandf = pd.read_csv(os.path.join(csv_path, 'clean_data.csv'))

In [5]:
cleandf

,Unnamed: 0,text,target
0,0,smell hillary fear daniel greenfield shillman ...,0
1,1,watch exact moment paul ryan commit political ...,0
2,2,kerry go paris gesture sympathy u secretary st...,1
3,3,bernie supporter twitter erupt anger dnc try w...,0
4,4,battle new york primary matter primary day new...,1
...,...,...,...
6330,6330,state department say can find email clinton sp...,1
6331,6331,p pb stand plutocratic pentagon p pb stand plu...,0
6332,6332,anti trump protester tool oligarchy informatio...,0
6333,6333,ethiopia obama seek progress peace security ea...,1


In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Embedding, Flatten, Masking, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pickle

2022-08-29 12:10:05.798355: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-29 12:10:05.798434: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
tokenizer = Tokenizer()
model = Sequential()
es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
X_train, X_test, y_train, y_test = train_test_split(cleandf['text'], cleandf['target'], test_size=0.2)

In [6]:
tokenizer.fit_on_texts(X_train)

In [11]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)

In [20]:
X_train_token = pad_sequences(X_train_token, dtype='int32', padding='post', maxlen=300)
X_test_token = pad_sequences(X_test_token, dtype='int32', padding='post', maxlen=300)

In [23]:
model.add(Embedding(
    input_dim=(len(tokenizer.word_index))+1, # +1 for the 0 padding
    input_length=300, # Max_sentence_length (optional, for model summary)
    output_dim=100,
    mask_zero=True, # Built-in masking layer :)
))
#model.add(GRU(20, return_sequences=True))
model.add(Conv1D(30, kernel_size=5, activation='tanh'))
model.add(Flatten())
model.add(Dense(15, activation='relu'))
#model.add(Dropout(rate=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer='rmsprop',
             metrics=['accuracy'])

2022-08-29 12:52:43.862383: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 19795200 exceeds 10% of free system memory.
2022-08-29 12:52:43.970381: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 19795200 exceeds 10% of free system memory.
2022-08-29 12:52:43.985310: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 19795200 exceeds 10% of free system memory.


In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 100)          4960900   
                                                                 
 conv1d (Conv1D)             (None, 296, 30)           15030     
                                                                 
 flatten (Flatten)           (None, 8880)              0         
                                                                 
 dense (Dense)               (None, 15)                133215    
                                                                 
 dense_1 (Dense)             (None, 1)                 16        
                                                                 
Total params: 5,109,161
Trainable params: 5,109,161
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(X_train_token, y_train, 
          epochs=50, 
          batch_size=32,
          validation_split=0.2,
          callbacks=[es],
        use_multiprocessing=True
         )

Epoch 1/50


2022-08-29 12:52:59.127673: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 19795200 exceeds 10% of free system memory.
2022-08-29 12:53:02.247560: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 19795200 exceeds 10% of free system memory.


127/127 [==============================] - 44s 303ms/step - loss: 0.4103 - accuracy: 0.7997 - val_loss: 0.2454 - val_accuracy: 0.8935
Epoch 2/50
127/127 [==============================] - 17s 135ms/step - loss: 0.1128 - accuracy: 0.9583 - val_loss: 0.2669 - val_accuracy: 0.8945
Epoch 3/50
 92/127 [====================>.........] - ETA: 3s - loss: 0.0232 - accuracy: 0.9946

KeyboardInterrupt: 

In [13]:
model.evaluate(X_test_token, y_test, verbose=1)

40/40 [==============================] - 0s 5ms/step - loss: 0.2481 - accuracy: 0.8958


[0.24811822175979614, 0.8958168625831604]

In [26]:
model.predict(X_test_token[1].reshape(1,300))

array([[0.9999995]], dtype=float32)

In [28]:
model.predict(X_test_token[1].reshape(1,300))

array([[0.9999995]], dtype=float32)

In [32]:
y_pred = model.predict(X_test_token[1].reshape(1, 300))

y_pred[0][0]


0.9999995

# Attention: Beggining of Word2Vec
## Better run only on Google Colab

In [8]:
from gensim.models import Word2Vec
import numpy as np
import gensim.downloader as api

In [9]:
#word2vec = api.load('glove-wiki-gigaword-50') # this one crashes my kernel everytime
word2vec = api.load('text8') # this one takes an eternity

[==================================================] 100.0% 31.6/31.6MB downloaded


In [10]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

In [11]:
# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

In [12]:
# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)

KeyboardInterrupt: 

In [ ]:
X_train_embed = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=300)
X_test_embed = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=300)

In [ ]:
for X in [X_train_embed, X_test_embed]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec.vector_size


assert X_train_embed.shape[0] == len(X_train)
assert X_test_embed.shape[0] == len(X_test)

In [ ]:
model = Sequential()
model.add(Masking(mask_value=0, input_shape=X_train_embed.shape[1:]))
#model.add(LSTM(20, return_sequences=True))
model.add(Conv1D(30, kernel_size=5, activation='tanh'))
model.add(Flatten())
model.add(Dense(15, activation='relu'))
#model.add(Dropout(rate=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer='rmsprop',
             metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train_embed, y_train, 
          epochs=50, 
          batch_size=32,
          validation_split=0.2,
          callbacks=[es],
        use_multiprocessing=True
         )

In [ ]:
model.evaluate(X_test_embed, y_test, verbose=1)